# Trabajo 4

In [6]:
#Importaciones
import pandas as pd
import numpy as np

### Cargo todos los CSV 

In [7]:
df_materias=pd.read_csv("Data\Alu_Materia.csv",sep=';')
df_carreras=pd.read_csv("Data\Alu_Carrera.csv",sep=';')
info_materias=pd.read_csv("Data\Materias.csv",sep=';')
info_finales=pd.read_csv("Data\Finales.csv",sep=';')
info_carreras=pd.read_csv("Data\Carreras.csv",sep=';')
#df_carreras['Fecha_Nac']
#df_materias

In [8]:
#df_materias.info()
#df_carreras.info()
#info_carreras.info()
#info_finales.info()
#info_materias.info()

### Arreglo de datos

In [9]:
#Dropeo esas columnas que no me sirven para este analisis
#Y elimino registros sobrantes para el analisis, todos aquellos menores a 2000 y mayores al año 2010
df_materias=df_materias.drop(['Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup','Ctro_Ext','Id_Comision'],axis=1)
df_materias=df_materias[(df_materias.Año>2000)&(df_materias.Año<2011)]

In [10]:
#Elimino columnas que no me sirven
df_carreras=df_carreras.drop(['Id_Ctro_Ext' ,'Id_Modalidad' ,'Fecha_Ingreso' ,'Activo'],axis=1)

In [11]:
#Elimino datos con ruido, duplicados 
info_finales=info_finales.drop_duplicates(subset=info_finales.columns.tolist()).reset_index().drop(['index'],axis=1)

In [12]:
#Elimino columnas que no me sirven
info_carreras=info_carreras.drop(['Nombre_Largo'],axis=1)

### Merge

In [13]:
#entre df materias y df_carreras
merge1=df_carreras.merge(df_materias,on=['Id_Carrera','Id_Alumno'])


In [14]:
#Entre el primer merge e info_finales
merge2=merge1.merge(info_finales,how='left',right_on=['Id_Alumno', 'Id_Carrera', 'Id_Materia', 'Semetre_Cursada','Año_Cursada'], 
                    left_on=['Id_Alumno', 'Id_Carrera', 'Id_Materia', 'Semestre','Año'])
#merge2

In [15]:
#merge con el 2 e info_carreras
merge3=merge2.merge(info_carreras,on=['Id_Carrera'])
#merge3

In [16]:
#Merge 3 con info_materias
merge4=merge3.merge(info_materias,on=['Id_Materia'])
merge4.columns

Index(['Id_Alumno', 'Id_Carrera', 'Semestre_Ingreso', 'Año_Ingreso', 'Sexo',
       'Fecha_Nac', 'Id_Materia', 'Semestre', 'Año', 'Fecha_Inscrip',
       'Id_Modalidad', 'Nota_1p', 'Nota_2p', 'Nota_TP', 'Nota_Recup',
       'Cuatrimestre', 'Semetre_Cursada', 'Año_Cursada', 'Fecha_Examen',
       'Nota', 'Fecha_Inscrip_Examen', 'Nombre_Corto', 'Abreviatura',
       'Cuatrimestres', 'Nombre'],
      dtype='object')

### Arreglo Datos totales

In [17]:
#Reemplazo valores del ausente (11) por nan para que no afecte en el analisis. 
#Lo aplico sobre las columnas de las notas
merge4['Nota_1p'] = merge4['Nota_1p'].replace(11.0,np.NaN)
merge4['Nota_2p'] = merge4['Nota_2p'].replace(11.0,np.NaN)
merge4['Nota_TP'] = merge4['Nota_TP'].replace(11.0,np.NaN)
merge4['Nota_Recup'] = merge4['Nota_Recup'].replace(11.0,np.NaN)

In [18]:
#Elimino la columna Año_cursada y semestre_cursada ya que se repiten los datos de Año y Semetre del df_materias
merge4=merge4.drop(['Año_Cursada','Semetre_Cursada'],axis=1)

In [19]:
#Convierto datos de fechas que son objects a datetime
merge4['Fecha_Nac']=pd.to_datetime(merge4['Fecha_Nac']).dt.year
merge4['Fecha_Examen']=pd.to_datetime(merge4['Fecha_Examen']).dt.floor('d')
merge4['Fecha_Inscrip_Examen']=pd.to_datetime(merge4['Fecha_Inscrip_Examen']).dt.floor('d')
merge4['Fecha_Inscrip']=pd.to_datetime(merge4['Fecha_Inscrip']).dt.floor('d')

In [20]:
#Cambio nombre de columnas
merge4=merge4.rename(columns={'Semestre':'Semest_Cursada','Año':'Año_Cursada','Cuatrimestre':'Cuatri_Materia',
                      'Nota':'Nota_Final','Nombre_Corto':'Name_Carrera','Abreviatura':'Abrev_Car',
                      'Cuatrimestres':'Cuatris_Carrera','Nombre':'Name_Materia'})

### Clasificacion notas

In [21]:
#Clasificacion

def ClasificAcademy(p1,p2,tp,rec):
#limpieza de datos. 

    if p1 ==11 : p1 = np.nan
    if p2 ==11 : p2 = np.nan
    if tp ==11 : tp = np.nan 
    if rec ==11 : rec = np.nan
        
        
    #Clasificacion de alumnos. 
    if (p1 >= 7) and (p2 >= 7) and (tp >= 7):
        return 'P'
    elif (p1 >= 5) and (p2 >= 5) and (tp >= 5):
        return 'R'
    elif (tp >= 5) and (rec >= 5):
        return 'R'
    elif (tp == 11) or (np.isnan(tp)):
        return 'D'
    elif (rec == 11) or (np.isnan(rec)):
        return 'D'
    else:
        return 'L'


In [22]:
#funcion para agregar la columna y los datos de la condicion
merge4["Cond_Materia"]=merge4.apply(lambda f:ClasificAcademy(f['Nota_1p'],f['Nota_2p'],f['Nota_TP'],f['Nota_Recup']),axis=1 )

In [23]:
#Calsificador del final. 
#'Ap'= aprobado
#'Rep'= reprobado
#'Aus'= ausente
def ClasificFinal(nota):
    if nota >= 4 and nota <11:
        return "Ap"
    if nota < 4:
        return "Rep"
    else:
        return "Aus"
    


In [24]:
merge4["Cond_Final"]=merge4.apply(lambda f:ClasificFinal(f['Nota_Final']),axis=1 )

In [25]:
#Creo nueva columna con las edades
merge4['Edad']=merge4['Año_Ingreso']-merge4['Fecha_Nac']

In [26]:
sorted(merge4['Edad'].unique())
#Cambiar por nan las fechas de nac menores a 17

[-18,
 -11,
 -9,
 -8,
 -7,
 -6,
 -5,
 -4,
 -3,
 3,
 14,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 42,
 43,
 48,
 52,
 53]

In [27]:
#Modifico datos rotos, aquellos donde la edad era menor a 17 años. No elimino el dato sino que reemplazo por nan
merge4['Edad'] = np.where(merge4['Edad'] < 17, np.nan, merge4['Edad'])

### DF a csv

In [28]:
merge4.to_csv('ies_todos.csv',index=False)